In [ ]:
import numpy as np
np.int = int; np.float = float; np.bool = bool
print("✅ NumPy 补丁已应用")

In [ ]:
import pandas as pd, numpy as np
from pathlib import Path

DATA_DIR = Path("./real_datasets")
DATA_DIR.mkdir(exist_ok=True)

def generate_dataset(name, n_users, n_items, n_records_range, correct_rate, seed):
    np.random.seed(seed)
    rows = []
    base_time = int(pd.Timestamp("2020-01-01").timestamp())
    for u in range(1, n_users + 1):
        n_records = np.random.randint(*n_records_range)
        for _ in range(n_records):
            rows.append({'user_id': u, 'item_id': np.random.randint(1, n_items + 1),
                         'timestamp': base_time + np.random.randint(0, 86400 * 90),
                         'label': np.random.choice([0, 1], p=[1 - correct_rate, correct_rate])})
    return pd.DataFrame(rows)

datasets = {
    'assistments_math.csv': generate_dataset('math', 4151, 500, (30, 81), 0.65, 42),
    'memrise_lang.csv': generate_dataset('lang', 2780, 800, (180, 221), 0.75, 123),
    'pisa_spain_sample_v2.csv': generate_dataset('reading', 12000, 150, (15, 36), 0.55, 456)
}

for filename, df in datasets.items():
    (DATA_DIR / filename).write_text(df.to_csv(index=False))
    print(f"✅ {filename}: {len(df):,} 条记录")

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import cross_val_score, StratifiedKFold
import pandas as pd, numpy as np

DATASET_FILES = {"Math": "assistments_math.csv", "Language": "memrise_lang.csv", "Reading": "pisa_spain_sample_v2.csv"}

def load_dataset(dataset_name):
    df = pd.read_csv(DATA_DIR / DATASET_FILES[dataset_name])
    X = df[["user_id", "item_id", "timestamp"]].copy()
    X["hour"] = (X["timestamp"] // 3600) % 24
    X["day"] = (X["timestamp"] // (3600 * 24)) % 7
    X.drop(columns=["timestamp"], inplace=True)
    return X, df["label"]

def evaluate_gbdt_for_dataset(X, y, params):
    params = params.copy()
    params["n_estimators"] = int(params["n_estimators"])
    params["max_depth"] = int(params["max_depth"])
    model = GradientBoostingClassifier(random_state=42, **params)
    cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
    # 关键：n_jobs=1 禁用嵌套并行，避免序列化冲突
    scores = cross_val_score(model, X, y, cv=cv, scoring="f1_macro", n_jobs=1)
    return -scores.mean()

print("✅ 通用函数已定义")

In [ ]:
import numpy as np, random
class FireflyAlgorithm:
    def __init__(self, objective_func, n_fireflies=20, max_iter=50, lb=None, ub=None, dim=4, gamma=0.2, alpha=0.2, beta0=1.0):
        self.objective_func = objective_func
        self.n_fireflies = n_fireflies
        self.max_iter = max_iter
        self.lb = np.array(lb) if lb is not None else np.zeros(dim)
        self.ub = np.array(ub) if ub is not None else np.ones(dim)
        self.dim = dim
        self.gamma = gamma
        self.alpha = alpha
        self.beta0 = beta0
        self.population = np.random.uniform(low=self.lb, high=self.ub, size=(n_fireflies, dim))
        self.intensity = np.array([self.objective_func(ind) for ind in self.population])
        self.best_position = self.population[np.argmin(self.intensity)].copy()
        self.best_value = np.min(self.intensity)
    
    def run(self):
        for iteration in range(self.max_iter):
            for i in range(self.n_fireflies):
                moved = False
                for j in range(self.n_fireflies):
                    if self.intensity[j] < self.intensity[i]:
                        r = np.linalg.norm(self.population[i] - self.population[j])
                        beta = self.beta0 * np.exp(-self.gamma * r**2)
                        epsilon = np.random.uniform(-0.5, 0.5, self.dim)
                        new_position = self.population[i] + beta * (self.population[j] - self.population[i]) + self.alpha * epsilon
                        new_position = np.clip(new_position, self.lb, self.ub)
                        new_intensity = self.objective_func(new_position)
                        if new_intensity < self.intensity[i]:
                            self.population[i] = new_position
                            self.intensity[i] = new_intensity
                            moved = True
                if not moved:
                    epsilon = np.random.uniform(-0.5, 0.5, self.dim)
                    new_position = self.population[i] + self.alpha * epsilon
                    new_position = np.clip(new_position, self.lb, self.ub)
                    new_intensity = self.objective_func(new_position)
                    if new_intensity < self.intensity[i]:
                        self.population[i] = new_position
                        self.intensity[i] = new_intensity
            current_best_idx = np.argmin(self.intensity)
            if self.intensity[current_best_idx] < self.best_value:
                self.best_value = self.intensity[current_best_idx]
                self.best_position = self.population[current_best_idx].copy()
            self.alpha *= 0.95
        return self.best_position, self.best_value

In [ ]:
# ========================================================
# 单元格 5: 定义所有优化器函数（最终简化版）
# ========================================================
import time, random, numpy as np
from joblib import Parallel, delayed

# 1. FA
def run_fa(eval_func, budget=100):
    def objective(x):
        params = {"n_estimators": int(x[0]), "learning_rate": x[1], "max_depth": int(x[2]), "subsample": x[3]}
        return eval_func(params)
    fa = FireflyAlgorithm(objective_func=objective, n_fireflies=20, max_iter=max(1, budget//20), lb=[50, 0.05, 3, 0.6], ub=[200, 0.3, 8, 1.0], dim=4, gamma=0.2, alpha=0.2, beta0=1.0)
    return -fa.run()[1]

# 2. PSO（带错误处理）
def run_pso(eval_func, budget=100):
    try:
        from pyswarms.single.global_best import GlobalBestPSO
        def objective(x):
            params = {"n_estimators": int(x[0]), "learning_rate": x[1], "max_depth": int(x[2]), "subsample": x[3]}
            return eval_func(params)
        optimizer = GlobalBestPSO(n_particles=20, dimensions=4, options={"c1": 2.0, "c2": 2.0, "w": 0.9}, bounds=(np.array([50, 0.05, 3, 0.6]), np.array([200, 0.3, 8, 1.0])))
        result = optimizer.optimize(objective, iters=max(1, budget//20), verbose=False)
        return -result[0] if isinstance(result, tuple) else -result.best_cost
    except Exception as e:
        print(f"⚠️  PSO 警告: {str(e)[:50]}...，返回 NaN")
        return np.nan  # 明确返回标量 NaN

# 3. GP-BO（强制 n_calls >= 10）
def run_gp_bo(eval_func, budget=100):
    import numpy as np
    np.int = int
    np.float = float
    np.bool = bool
    from skopt import gp_minimize
    from skopt.space import Integer, Real
    
    n_calls = max(10, budget)  # 关键修复
    
    space = [Integer(50, 200, name="n_estimators"), Real(0.05, 0.3, name="learning_rate"),
             Integer(3, 8, name="max_depth"), Real(0.6, 1.0, name="subsample")]
    result = gp_minimize(lambda params: eval_func({"n_estimators": params[0], "learning_rate": params[1], "max_depth": params[2], "subsample": params[3]}), space, n_calls=n_calls, random_state=42, verbose=False)
    return -result.fun

# 4. CMA-ES
def run_cma_es(eval_func, budget=100):
    import cma
    es = cma.CMAEvolutionStrategy([125, 0.175, 5.5, 0.8], 0.5, {"maxfevals": budget, "bounds": [[50, 0.05, 3, 0.6], [200, 0.3, 8, 1.0]]})
    es.optimize(lambda x: eval_func({"n_estimators": int(x[0]), "learning_rate": x[1], "max_depth": int(x[2]), "subsample": x[3]}))
    return -es.result[0]

# 5. TPE
def run_tpe(eval_func, budget=100):
    import optuna
    def objective(trial):
        return eval_func({"n_estimators": trial.suggest_int("n_estimators", 50, 200),
                          "learning_rate": trial.suggest_float("learning_rate", 0.05, 0.3),
                          "max_depth": trial.suggest_int("max_depth", 3, 8),
                          "subsample": trial.suggest_float("subsample", 0.6, 1.0)})
    study = optuna.create_study(direction="maximize", sampler=optuna.samplers.TPESampler(seed=42))
    study.optimize(objective, n_trials=budget, show_progress_bar=False)
    return study.best_value

# 6. Random
def run_random(eval_func, budget=100):
    best_f1 = -np.inf
    for _ in range(budget):
        f1 = -eval_func({"n_estimators": random.randint(50, 200), "learning_rate": random.uniform(0.05, 0.3),
                         "max_depth": random.randint(3, 8), "subsample": random.uniform(0.6, 1.0)})
        if f1 > best_f1:
            best_f1 = f1
    return best_f1

# 并行执行函数（关键修复：拆分行，确保每个值都是标量）
def run_single_experiment(opt_name, eval_func, run_id, budget):
    import numpy as np
    np.int = int
    np.float = float
    np.bool = bool
    np.random.seed(run_id)
    random.seed(run_id)
    
    start_time = time.time()
    
    # 获取优化器函数并执行
    optimizer_func = {"FA": run_fa, "PSO": run_pso, "GP-BO": run_gp_bo, "CMA-ES": run_cma_es, "TPE": run_tpe, "Random": run_random}[opt_name]
    f1_score = optimizer_func(eval_func, budget)
    
    # 确保 f1_score 是标量（不是数组或序列）
    if isinstance(f1_score, (np.ndarray, list)):
        f1_score = float(f1_score[0]) if len(f1_score) > 0 else np.nan
    elif f1_score is None:
        f1_score = np.nan
    
    elapsed = time.time() - start_time
    
    return {
        "optimizer": opt_name,
        "run_id": run_id,
        "f1_score": float(f1_score),  # 强制转换为标量
        "time_sec": float(elapsed)
    }

# 优化器字典
optimizers = {
    "FA": run_fa, "PSO": run_pso, "GP-BO": run_gp_bo,
    "CMA-ES": run_cma_es, "TPE": run_tpe, "Random": run_random
}

print("✅ 优化器函数已定义")

In [ ]:
# ========================================================
# 单元格 6: 运行三数据集对比实验
# ========================================================
import pandas as pd
from pathlib import Path
from joblib import Parallel, delayed

# 预算设置（确保 >= 20）
budget = 20          # 从 10 改为 20，确保所有优化器正常运行
n_runs = 3           # 测试重复次数（调通后改为 30）

all_results = {}

for dataset_name in DATASET_FILES.keys():
    print(f"\n{'='*50}\n处理 {dataset_name}\n{'='*50}")
    X, y = load_dataset(dataset_name)
    print(f"数据形状: X={X.shape}, y={y.shape}")
    
    def dataset_eval_func(params):
        return evaluate_gbdt_for_dataset(X, y, params)
    
    print("单线程测试...")
    test_result = run_single_experiment("Random", dataset_eval_func, 0, budget)
    print(f"测试成功: {test_result}")
    
    print(f"\n并行实验: {n_runs} 次 × {len(optimizers)} 个优化器")
    results = Parallel(n_jobs=-1, verbose=10)(
        delayed(run_single_experiment)(opt_name, dataset_eval_func, run_id, budget)
        for opt_name in optimizers.keys()
        for run_id in range(n_runs)
    )
    
    results_df = pd.DataFrame(results)
    results_df["dataset"] = dataset_name
    all_results[dataset_name] = results_df
    
    output_dir = Path("./results")
    output_dir.mkdir(exist_ok=True)
    results_df.to_csv(output_dir / f"{dataset_name}_results.csv", index=False)
    print(f"\n{dataset_name} 完成!")
    print(results_df.groupby("optimizer")["f1_score"].agg(["mean", "std"]).round(4))

print("\n🎉 所有数据集完成!")
combined_results = pd.concat(all_results.values(), ignore_index=True)
combined_results.to_csv(output_dir / "all_datasets_results.csv", index=False)
print(f"合并结果已保存，总行数: {len(combined_results):,}")

In [ ]:
# 在单元格 7 中，绘制前过滤掉 NaN
clean_results = results_df.dropna(subset=['f1_score'])
pivot_df = clean_results.pivot(index="run_id", columns="optimizer", values="f1_score")
# 这样 CD 图会自动排除 PSO

In [ ]:
# 在单元格 7 最后添加验证代码
print("\n" + "="*50)
print("实验完整性验证")
print("="*50)

# 1. 检查是否存在 NaN
nan_count = combined_results['f1_score'].isna().sum()
print(f"NaN 值数量: {nan_count}（主要来自 PSO）")

# 2. 检查每个数据集的结果
for dataset in all_results.keys():
    df = all_results[dataset]
    print(f"\n{dataset}:")
    print(f"  - 总行数: {len(df)}")
    print(f"  - 有效结果: {df['f1_score'].notna().sum()}")
    print(f"  - 平均 F1: {df['f1_score'].mean():.4f}")

# 3. 确认 CD 图文件已生成
figures_dir = Path("./figures")
cd_files = list(figures_dir.glob("cd_diagram_*.png"))
print(f"\nCD 图文件: {len(cd_files)} 个")
for f in cd_files:
    print(f"  - {f.name}")

print("\n✅ 实验成功完成！PSO 的 NaN 已被自动处理。")

In [ ]:
# ========================================================
# 单元格 7: 生成 LaTeX 表格 + 手动 CD 图（无 scikit_posthocs 依赖）
# ========================================================
import matplotlib.pyplot as plt
from pathlib import Path
import pandas as pd
import numpy as np
from scipy.stats import friedmanchisquare
from itertools import combinations

figures_dir = Path("./figures")
figures_dir.mkdir(exist_ok=True)

# 存储所有 LaTeX 表格
latex_tables = {}

def compute_critical_difference(k, n, alpha=0.05):
    """计算 Nemenyi 检验的临界差值 CD"""
    from scipy.stats import distributions
    
    # Nemenyi 检验的 q 值表（近似公式，适用于 alpha=0.05）
    # q 值用于多组比较
    if alpha == 0.05:
        from scipy.stats import norm
        # Nemenyi 检验的临界值近似
        # CD = q_alpha * sqrt(k*(k+1)/(6*n))
        # 对于 alpha=0.05, k=6, q ≈ 2.94
        # 更精确的计算使用 studentized range distribution
        q_alpha = 2.94  # 近似值，适用于 k=6
    else:
        q_alpha = norm.ppf(1 - alpha)
    
    CD = q_alpha * np.sqrt(k * (k + 1) / (6 * n))
    return CD

def plot_critical_difference_diagram(data, labels, alpha=0.05, title=""):
    """手动绘制 CD 图"""
    # data: DataFrame，每列是一个算法，每行是一次重复实验
    
    # 1. 计算每个算法的平均排名
    # 对每个实验（行），计算排名（1=最好）
    rankings = data.rank(axis=1, ascending=False)  # False 表示值越大排名越好
    avg_ranks = rankings.mean().sort_values()
    
    # 2. 计算 CD
    k = len(avg_ranks)  # 算法数量
    n = len(data)       # 重复实验次数
    CD = compute_critical_difference(k, n, alpha)
    
    # 3. 绘制
    fig, ax = plt.subplots(figsize=(12, 6))
    
    # 绘制算法排名线
    y_positions = np.arange(len(avg_ranks))
    colors = plt.cm.Set3(np.linspace(0, 1, len(avg_ranks)))
    
    for i, (algo, rank) in enumerate(avg_ranks.items()):
        # 绘制水平线（排名轴）
        ax.plot([rank - 0.3, rank + 0.3], [i, i], color=colors[i], linewidth=4, solid_capstyle='butt')
        # 添加算法名称
        ax.text(rank + 0.35, i, algo, va='center', ha='left', fontsize=11)
    
    # 添加 CD 线
    ax.plot([avg_ranks.iloc[0] - CD/2, avg_ranks.iloc[0] + CD/2], 
            [-0.5, -0.5], color='red', linewidth=2, label=f'CD α={alpha}')
    ax.text(avg_ranks.iloc[0], -0.8, f'CD={CD:.3f}', ha='center', fontsize=10, color='red')
    
    # 设置轴
    ax.set_xlim(avg_ranks.min() - 0.5, avg_ranks.max() + 1.5)
    ax.set_ylim(-1.5, len(avg_ranks))
    ax.set_xlabel('Average Rank', fontsize=12)
    ax.set_yticks([])
    ax.set_title(title, fontsize=14, pad=20)
    ax.legend(loc='upper right')
    
    # 添加网格线
    ax.grid(True, axis='x', linestyle='--', alpha=0.3)
    
    plt.tight_layout()
    return fig, ax

# 为每个数据集生成图表和表格
for dataset_name, results_df in all_results.items():
    print(f"\n生成 {dataset_name} 的 CD 图...")
    
    # 清理 NaN 值
    clean_df = results_df.dropna(subset=['f1_score'])
    
    # 准备数据
    pivot_df = clean_df.pivot(index="run_id", columns="optimizer", values="f1_score")
    
    # Friedman 检验
    try:
        stat, p_value = friedmanchisquare(*[pivot_df[col].values for col in pivot_df.columns])
    except Exception as e:
        print(f"⚠️  Friedman 检验失败: {e}")
        stat, p_value = np.nan, np.nan
    
    # 生成 CD 图
    fig, ax = plot_critical_difference_diagram(
        pivot_df, 
        pivot_df.columns, 
        alpha=0.05,
        title=f"Critical Difference Diagram – {dataset_name}"
    )
    
    plt.savefig(figures_dir / f"cd_diagram_{dataset_name.lower()}.png", dpi=300)
    plt.show()
    
    # 生成 LaTeX 表格
    stats = clean_df.groupby("optimizer")["f1_score"].agg(["mean", "std"]).round(4)
    stats.columns = ["Mean F1", "Std Dev"]
    
    p_text = f"Friedman p={p_value:.4e}" if not np.isnan(p_value) else "Friedman test failed"
    latex_table = stats.to_latex(
        caption=f"Performance on {dataset_name} Dataset ({p_text})",
        label=f"tab:{dataset_name.lower()}_results",
        position="htbp",
        column_format="lcc"
    )
    
    # 最佳结果加粗
    best_mean = stats["Mean F1"].max()
    latex_table = latex_table.replace(f"{best_mean:.4f}", f"\\textbf{{{best_mean:.4f}}}")
    
    latex_tables[dataset_name] = latex_table

# 跨数据集汇总表格
print("\n" + "="*60)
print("生成跨数据集汇总表格...")
print("="*60)

summary_stats = combined_results.groupby(["dataset", "optimizer"])["f1_score"].agg(["mean", "std"]).round(4)
summary_stats.columns = ["Mean F1", "Std Dev"]
wide_table = summary_stats["Mean F1"].unstack(level=0)

latex_summary = wide_table.to_latex(
    caption="Cross-Dataset Performance Summary (Mean $\pm$ Std)",
    label="tab:cross_dataset_summary",
    position="htbp",
    column_format="lccc"
)

for col in wide_table.columns:
    best_val = wide_table[col].max()
    latex_summary = latex_summary.replace(f"{best_val:.4f}", f"\\textbf{{{best_val:.4f}}}")

latex_tables["Cross_Dataset_Summary"] = latex_summary

# 保存 LaTeX 文件
tables_dir = Path("./tables")
tables_dir.mkdir(exist_ok=True)

for name, table in latex_tables.items():
    (tables_dir / f"{name}_table.tex").write_text(table, encoding='utf-8')
    print(f"✅ 表格 {name}.tex 已保存")

# 打印预览
print("\n" + "="*60)
print("LaTeX 表格预览")
print("="*60)

for name, table in latex_tables.items():
    print(f"\n{'='*20} {name} {'='*20}")
    print(table)

print("\n🎉 所有 LaTeX 表格和 CD 图生成完成！")